# Create Meteorite Shapefiles

In [169]:
import os
import pandas as pd
import geopandas as gpd
import shapely

from zipfile import ZipFile

### Create a data frame

In [170]:
# Create a data frame from a CSV file
df = pd.read_csv("./assets/meteorite-data-nasa.csv")

### Wrangle the data

In [171]:
# Get rid of unnecessary columns
keep_cols = [
    "name",
    "mass (g)",
    "fall",
    "year",
    "reclat",
    "reclong"
]
df = df[keep_cols]

In [172]:
# Show percentage of rows that have null reclong or reclat values
(df["reclong"].isna() | df["reclat"].isna()).mean()

np.float64(0.16000962463907603)

In [173]:
# Keep only those rows where reclong nor reclat is null
df = df[(df["reclong"].notna() & df["reclat"].notna())]

In [174]:
# Now the null percentage is 0
(df["reclong"].isna() | df["reclat"].isna()).mean()

np.float64(0.0)

### Create a geo data frame

In [175]:
# Construct a geo data frame
geometry = shapely.points(df.pop("reclong"), df.pop("reclat"))
gdf = gpd.GeoDataFrame(df, geometry = geometry)

In [ ]:
# Set its Coordinate Reference System
gdf = gdf.geometry.set_crs("EPSG:4326")

In [180]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [177]:
# Narrow meteorites down to only those that where witnessed
gdf = gdf[gdf["fall"] == "Fell"]

### Create a Shapefile

In [178]:
# Create a Shapefile from the geo data frame
if not os.path.isdir("./shapefiles"):
    os.mkdir("./shapefiles")
gdf.to_file("./shapefiles/meteorites.shp")

In [179]:
# Compress the Shapefile parts into a ZIP file, simultaneously removing them
with ZipFile("./shapefiles/meteorites.zip", "w") as zipf:
    for file in os.listdir(os.fsencode("./shapefiles/")):
      filename = os.fsdecode(file)
      if not filename.endswith(".zip"):
        zipf.write("./shapefiles/" + filename)
        os.remove("./shapefiles/" + filename)